# Базовые сведения о процессах

У процесса бывают следующие характеристики:

* Идентификатор процесса.  
* Объем оперативной памяти.  
* Стек.  
* Список открытых файлов.  
* Ввод/вывод.  

Посмотрим запущенные на текущий момент процессы:

In [1]:
%%bash

top -b -n 1

top - 20:58:30 up 11 min,  0 users,  load average: 0.52, 0.58, 0.59
Tasks:   6 total,   1 running,   5 sleeping,   0 stopped,   0 zombie
%Cpu(s): 14.1 us, 11.6 sy,  0.0 ni, 73.5 id,  0.0 wa,  0.9 hi,  0.0 si,  0.0 st
KiB Mem :  8288672 total,  3725272 free,  4326924 used,   236476 buff/cache
KiB Swap: 24361084 total, 24282808 free,    78276 used.  3820892 avail Mem 

  PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND
    1 root      20   0    8892    308    272 S   0.0  0.0   0:00.12 init
    5 root      20   0    8916    232    180 S   0.0  0.0   0:00.01 init
    6 cyril     20   0   15316   3972   3708 S   0.0  0.0   0:00.37 bash
  416 root      20   0    8916    232    180 S   0.0  0.0   0:00.00 init
  417 cyril     20   0   13636   1872   1768 S   0.0  0.0   0:00.08 bash
  433 cyril     20   0   15768   1796   1324 R   0.0  0.0   0:00.01 top


Теперь создадим наш Python-процесс и посмотрим на его характеристики средствами операционной системы, после чего остановим его.

In [2]:
%%writefile sleep.py

import os
import time

pid = os.getpid()

while True:
    print(pid, time.time())
    time.sleep(2)

Overwriting sleep.py


In [3]:
%%bash

python3 sleep.py &

#top -b -n 1 | grep python3
ps aux | head -1; ps axu | grep "sleep.py"
top -b -n 1 | grep python3 | awk '{print $1}' | xargs -r kill -9

USER       PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
cyril      451  0.0  0.0  21168  5352 tty2     S    20:58   0:00 python3 sleep.py
cyril      455  0.0  0.0  12892  1124 tty2     S    20:58   0:00 grep sleep.py


-bash: line 6:   451 Killed                  python3 sleep.py


При своей работе программа выполняет системные вызовы (что логично). Эти вызовы можно отслеживать командой `strace`:

`sudo strace -p <pid>`

Также программа может работать с разными файлами. Посмотреть их можно командой `lsof`:

`lsof -p <pid>`

# Программное создание процессов в Python

Сабж делается вызовом функции `os.fork()`. Посмотрим пример.

In [4]:
%%writefile child.py

import time
import os

pid = os.fork()

#Дочерний процесс
if pid == 0:
    for _ in range(3):
        print(f'child: {os.getpid()}; time: {time.time()}')
        time.sleep(2)
else:
    print(f'parent: {os.getpid()}')
    os.wait()

Overwriting child.py


In [5]:
%%bash

python3 child.py

child: 479; time: 1630778311.733288
child: 479; time: 1630778313.7338717
child: 479; time: 1630778315.7345257
parent: 478


Вызов `os.fork()`, как уже сказано, создает дочерний процесс. При этом дочернему процессу передаются все системные ресурсы, с которыми работает родительский процесс - копируются в т.ч. память и файловые дескрипторы. Также по коду можно понять, что функция `os.fork()` в дочерний процесс возвращает 0, а в родительский - `pid` дочернего процесса. А еще по коду можно понять, что вызов `os.wait()` дожидается завершения всех дочерних процессов. 

Если в bash мы хотим понять, какой процесс является родительским, а какой - дочерним, то нам поможет команда `ps -axf`.

Вызов функции `os.fork()` может рассматриваться лишь как иллюстрация создания дочерних процессов. На практике он не очень удобен и используют класс `Process` из модуля `multiprocessing`. Посмотрим, как это выглядит.

In [6]:
%%writefile no_fork.py

from multiprocessing import Process

def f(name: str):
    print(f'Hello {name}')
    
p = Process(target=f, args=('Bob',))
p.start()
p.join()

Overwriting no_fork.py


In [7]:
%%bash

python3 no_fork.py

Hello Bob


Еще более удобным является альтернативный способ использования класса `multiprocessing.Process`, когда мы создаем класс-наследник от `multiprocessing.Process`, передаем в инициализатор нужные для работы параметры и переопределяем метод `run()`:

In [8]:
%%writefile process_class.py

import sys
from multiprocessing import Process

class PrintProcess(Process):
    def __init__(self, name):
        super().__init__()
        self.name = name
        
    def run(self):
        print(f'hello {self.name}')
        
if __name__ == '__main__':
    p = PrintProcess(sys.argv[1])
    p.start()
    p.join()

Overwriting process_class.py


In [9]:
%%bash 

python3 process_class.py Cyril

hello Cyril


# Создание потоков в Python

Основное отличие потоков от процессов состоит в том, что потоки выполняются в рамках одного процесса, разделяя его память и другие ресурсы. При этом у каждого потока есть собственный стек. Посмотрим, как создать поток на Python.

In [10]:
%%writefile th.py

from threading import Thread

def f(name: str):
    print(f'Hello {name}!')
    
th = Thread(target=f, args=('Bob',))
th.start()
th.join

Overwriting th.py


In [11]:
%%bash

python3 th.py

Hello Bob!


Как и процесс, поток можно создать альтернативным способом - при помощи наследования.

In [12]:
%%writefile th_inh.py

import sys
from threading import Thread

class PrintThread(Thread):
    def __init__(self, name):
        super().__init__()
        self.name = name
        
    def run(self):
        print(f'Hello {self.name}')
        
if __name__ == '__main__':
    p = PrintThread(sys.argv[1])
    p.start()
    p.join()

Overwriting th_inh.py


In [13]:
%%bash

python3 th_inh.py Cyril

Hello Cyril


Также в Python3 есть очень интересная возможность запускать одну функцию с разными аргументами, используя фиксированное число потоков. Допустим, мы хотим многопоточно посчитать квадраты чисел от 0 до 9, используя не более 4 потоков.

In [14]:
%%writefile thread_pool.py

from concurrent.futures import ThreadPoolExecutor, as_completed

def f(a):
    return a * a

with ThreadPoolExecutor(max_workers=4) as pool:
    results = [pool.submit(f, i) for i in range(10)]
    
    for future in as_completed(results):
        print(future.result())

Overwriting thread_pool.py


In [15]:
%%bash

python3 thread_pool.py

36
64
0
25
9
4
81
1
16
49


# Синхронизация потоков и блокировки

Для потока можно организовать очередь задач. Как только место в этой очереди заканчивается, программа переходит в режим ожидания выполнения текущей задачи и новые задачи в очередь не пускает. На практике организация очереди задач выглядит вот так:

In [16]:
%%writefile queue_example.py

from queue import Queue
from threading import Thread

def worker(q: Queue, n: int):
    while True:
        item = q.get()
        if item is None:
            break
        print(f'process data: {n}, {item}')
        
q = Queue(5)
th1 = Thread(target=worker, args=(q, 1))
th2 = Thread(target=worker, args=(q, 2))
th2.start(); th1.start();

for i in range(5000):
    q.put(i)
    
q.put(None); q.put(None);
th1.join(); th2.join();

Overwriting queue_example.py


In [17]:
%%bash

python3 queue_example.py

process data: 2, 0
process data: 2, 1
process data: 2, 2
process data: 2, 3
process data: 2, 4
process data: 2, 5
process data: 2, 6
process data: 2, 7
process data: 2, 8
process data: 2, 9
process data: 2, 10
process data: 2, 11
process data: 2, 12
process data: 2, 13
process data: 2, 14
process data: 2, 15
process data: 2, 16
process data: 2, 17
process data: 2, 18
process data: 2, 19
process data: 2, 20
process data: 2, 21
process data: 2, 22
process data: 2, 23
process data: 2, 24
process data: 2, 25
process data: 2, 26
process data: 2, 27
process data: 2, 28
process data: 2, 29
process data: 2, 30
process data: 2, 31
process data: 2, 32
process data: 2, 33
process data: 2, 34
process data: 2, 35
process data: 2, 36
process data: 2, 37
process data: 2, 38
process data: 2, 39
process data: 2, 40
process data: 2, 41
process data: 2, 42
process data: 2, 43
process data: 2, 44
process data: 2, 45
process data: 2, 46
process data: 2, 47
process data: 2, 48
process data: 2, 49
process da

Очереди - это очень хороший механизм, и по возможности надо использовать его, избегая блокировок. К сожалению, иногда избежать блокировок не получается. Поэтому рассмотрим пример использования блокировок.

In [18]:
import threading

class Point(object):
    def __init__(self):
        self._mutex = threading.RLock()
        self._x = 0
        self._y = 0
        
    def get(self):
        with self._mutex:
            return (self._x, self._y)
        
    def set(self, x, y):
        with self._mutex:
            self._x, self._y = x, y

Как можно видеть, в коде выше регулярно применяется мьютекс. Он позволяет избежать проблемной ситуации, когда один поток установил у объекта координату x и не успел установить y, а другой поток забрал эти x и y, получив таким образом неконсистентное состояние. Такая ситуация называется гонкой за ресурсы (race condition). Рассмотрим пример более низкоуровневого использования мьютексов.

In [19]:
import threading

a = threading.RLock()
b = threading.RLock()

def foo():
    try:
        a.acquire()
        b.acquire()
    except:
        a.release()
        b.release()

Необходимо сказать, что вышеприведенный код не совсем корректен и приводит к дедлокам. Для их избежания надо освобождать ресурсы в обратной последовательности.

В Python есть еще один механизм синхронизации потоков. Называется он "Условные переменные". В качестве иллюстрации работы с условными переменными реализуем свой класс "Очередь", которая будет предназначена для работы с несколькими потоками.

In [20]:
%%writefile cond_var_example.py

import threading

class Queue(object):
    def __init__(self, size=5):
        self._size = size
        self._queue = []
        self._mutex = threading.RLock()
        self._empty = threading.Condition(self._mutex)
        self._full = threading.Condition(self._mutex)
        
    def put(self, val):
        with self._full:
            while len(self._queue) >= self._size:
                self._full.wait()
            self._queue.append(val)
            self._empty.notify()
            
    def get(self):
        with self._empty:
            while len(self._queue) == 0:
                self._empty.wait()
            result = self._queue.pop(0)
            self._full.notify()
            return result

Writing cond_var_example.py


Приведенного выше примера явно недостаточно для понимания работы условных переменных в Python. Позже необходимо копнуть глубже.

# Глобальная блокировка интерпретатора (GIL)

Основное предназначение GIL - это защита памяти интерпретатора от разрушений посредством представления всех операций с памятью в атомарном виде. Для начала рассмотрим в качетве примера программу, которая потребляет большой объем CPU. Эта программа запускает один и тот же набор операций сначала последовательно, а потом - параллельно, отображая время работы в каждом режиме.

In [22]:
import time
from threading import Thread

def count(n):
    while n > 0:
        n -= 1
        
# sequential run
t0 = time.time()
count(100_000_000)
count(100_000_000)
print(time.time() - t0)

# parallel run
t0 = time.time()
t1 = Thread(target=count, args=(100_000_000,))
t2 = Thread(target=count, args=(100_000_000,))
t1.start(); t2.start();
t1.join(); t2.join();
print(time.time() - t0)

13.935932636260986
11.950527429580688


Здесь видно, что время последовательного выполнения почти не отличается от времени параллельного выполнения. Отсюда вопрос: а какой вообще тогда смысл в распараллеливании? Ответ: в распараллеливании смысла нет, если распараллеливаемая задача нагружает процессор. В распараллеливании есть смысл, если задача, к примеру, использует большой объем ввода-вывода или часто обращается к системным вызовам.

GIL - это по сути объект класса `Threading.Lock`. Все потоки стараются захватить GIL. Исключением являются системные вызовы и операции ввода/вывода - для них GIL не нужен.